Importing necessary libraries

In [72]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import optimize
import pandas as pd
import glob
import math
import os

import multiprocessing
from multiprocessing import Pool

Given a dataset $X, Y$ each having size $n$ and a fitting function $f$, we define our fit error measure as -

$$J = \sum_{i=1}^{n} (y_i - f(x_i))^2$$

In [73]:
def fit_error(y, f):
    return np.sum((y-f)**2)

Fitting function 1, Gaussian - 

$$ p(z)= \frac{A}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(z-\mu)^2}{2 \sigma^2}}$$

Which we want to obey the same normalization as the numerical PDF -

$$ \int_{-\infty}^{\infty} p(z) dz = \sum_{i = 1}^{n} w_i$$

In [74]:
def fit_func_1(Z, A, mu, sigma):
    
    P = (A/np.sqrt(2*np.pi*sigma**2))*np.exp(-((Z-mu)**2/(2*sigma**2)))
    
    return P

Fitting function 2, normalized, piece-wise gaussian + power-law given by - 

$$ p(z)=  \left\{
\begin{array}{ll}
      \frac{A}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(z-\mu)^2}{2 \sigma^2}} : z < z_T \\
      B e^{- \alpha z} : z \ge z_T \\
\end{array} 
\right. $$

Enforcing continuity at $z = z_T$ yields -

$$ B = \frac{Ae^{\alpha z_T}}{\sqrt{2 \pi \sigma^2}} e^{- \frac{(z_T - \mu)^2}{2 \sigma^2}} $$

Further, enforcing normalization identical to the numerical PDF, we get -

$$A = \frac{\sum_{i = 1}^{n} w_i}{\frac{1}{2}\left( erf \left( \frac{z_T - \mu}{\sqrt{2} \sigma} \right) \right) + \frac{1}{\alpha \sqrt{2 \pi \sigma^2}} e^{- \frac{(z_T - \mu)^2}{2 \sigma^2}}} $$

Thus, the normalized PDF is -

$$ p(z)=  \frac{A}{\sqrt{2 \pi \sigma^2}} \left\{
\begin{array}{ll}
      e^{-\frac{(z-\mu)^2}{2 \sigma^2}} : z < z_T \\
      e^{-\frac{(z_T-\mu)^2}{2 \sigma^2}} \cdot e^{- \alpha(z-z_T)} : z \ge z_T \\
\end{array} 
\right. $$

Which obeys the same normalization as the numerical PDF -

$$ \int_{-\infty}^{\infty} p(z) dz = \sum_{i = 1}^{n} w_i $$

In [75]:
def fit_func_2(Z, A, mu, sigma, alpha, z_T):
    
    B = (A*np.exp(alpha*z_T))/np.sqrt(2*np.pi*sigma**2)*np.exp(-(z_T-mu)**2/(2*sigma**2))
    
    Z_1 = Z[np.where(Z <= z_T)]
    Z_2 = Z[np.where(Z > z_T)]
    
    P_1 = (A/np.sqrt(2*np.pi*sigma**2))*np.exp(-((Z_1 - mu)**2/(2*sigma**2)))
    P_2 = B*np.exp(-alpha*Z_2)

    P = np.concatenate([P_1, P_2])
    
    return P

Constructing fitted PDF

In [76]:
def fit_PDF(centers, heights):
            
    fit = np.zeros(len(centers))
    fit_err = 0
    fit_params = np.zeros(5)

    # Information about the peak in the numerical PDF
    peak_ind = np.where(heights == np.max(heights))[0][0]
    peak_height = np.max(heights)

    # mu is where the numerical PDF peaks
    mu = centers[peak_ind]

    # Estimating sigma using FWHM
    sigma = 0

    for i in range(0, peak_ind):
        if(heights[i] >= peak_height/2):
            sigma = (mu - centers[i])/np.sqrt(2*np.log(2))
            break

    # Estimating A accordingly, by using the peak value at mu
    A = np.sqrt(2*np.pi*sigma**2)*peak_height
    
    # First fit a Gaussian
    
    try:
        
        guess_params = np.array([A, mu, sigma])

        fit_params, fit_covar = optimize.curve_fit(fit_func_1, centers, heights, p0=guess_params, method = 'dogbox')
        fit = fit_func_1(centers, *fit_params)
        fit_err = fit_error(heights, fit)
        fit_params = np.concatenate([fit_params, np.array([float('nan'), float('nan')])])

        # See if an exponential decay tail exists and is a better fit

        for i in range(peak_ind, len(centers)):

            v_T = centers[i] 
            init_height = heights[i]

            # Estimating alpha using half-life decay
            alpha = 0

            for j in range(i+1, len(centers)):
                if(heights[j] <= init_height/2):
                    alpha = np.log(2)/(centers[j]-v_T)
                    break

            curr_guess_params = np.array([A, mu, sigma, alpha])

            curr_fit_params, curr_fit_covar = optimize.curve_fit(
                        lambda centers, A, mu, sigma, alpha: fit_func_2(centers, A, mu, sigma, alpha, v_T)
                            , centers, heights, p0=curr_guess_params)

            curr_fit = fit_func_2(centers, *curr_fit_params, v_T)
            curr_err = fit_error(heights, curr_fit)

            if(curr_err < fit_err):
                fit = curr_fit
                fit_err = curr_err
                fit_params = np.concatenate([curr_fit_params, np.array([v_T])])

    except:        
        
        pass

    return fit, fit_params

In [77]:
def fit_snap(info):
    
    snap_dict = info[0]
    metals = info[1]
    spath_metals = info[2]
    q = info[3]

    snap = snap_dict['snap']
    redshift = snap_dict['redshift']
        
    try:

        A = {}
        mu = {}
        sigma = {}
        alpha = {}
        z_T = {}

        for m in metals:

            num_df = pd.read_csv(spath_metals[m] +'data/num/' + str(snap) + '-num_' + m + '_data.csv')

            centers = np.array(num_df['abundance'].to_list())

            heights = np.array(num_df['num_val'].to_list())

            mass_norm = np.max(heights)

            heights /= mass_norm

            # Compute the fitted PDF

            fit, fit_params = fit_PDF(centers, heights)

            # Rescaling range to achieve desired normalization

            heights *= mass_norm
            fit *= mass_norm
            fit_params[0] *= mass_norm

            A[m] = fit_params[0]
            mu[m] = fit_params[1]
            sigma[m] = fit_params[2]
            alpha[m] = fit_params[3]
            z_T[m] = fit_params[4]

            datafile =  str(snap) + '-fit_' + m + '_data' + '.csv'

            fit_dict = {'abundance': centers, 'fit_val': fit}
            fit_df = pd.DataFrame(fit_dict)
            fit_df.to_csv(spath_metals[m] + 'data/fit/' + datafile, index = False)

        q.put({'snap_dict': snap_dict, 'A': A, 'mu': mu, 'sigma': sigma, 'alpha': alpha, 'z_T': z_T})
        
    except:
        
        for m in metals:
            
            datafile =  'fit_{}_params.csv'.format(m)
            fit_df = pd.read_csv(spath_metals[m] + 'data/fit/' + datafile)
            fit_df = fit_df[fit_df.snap != snap]
            fit_df.to_csv(spath_metals[m] + 'data/fit/' + datafile, index = False)

Importing dataset

In [78]:
# Specifying simulation directory and the directory to save results in
wdir = str(input('Enter simulation directory path: '))

# Specifying a snapshot for temporal analysis
sdir = wdir + 'temporal_analysis/'

Enter simulation directory path: /Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Code+Results/Results/m11b_res2100_cr_heating_fix/


In [79]:
# Get all rendered indices

rendered_df = pd.read_csv(sdir + 'rendered_snap_stats.csv')
snap_dicts = rendered_df.to_dict(orient = 'records')

# Get rendered metals

metal_df = pd.read_csv((sdir + 'metal_list.csv'))
metals = metal_df['metals'].to_list()

# Create a list of paths for all metals
spath_metals = {}

for m in metals:
    spath_metals[m] = sdir + m + '/'

In [80]:
if __name__ == "__main__":

    n_proc = multiprocessing.cpu_count()
    
    print("About to start:")
    
    manager = multiprocessing.Manager()
    q = manager.Queue()

    with Pool(processes = n_proc) as pool:

        pool.map(fit_snap, [(snap_dict, metals, spath_metals, q) for snap_dict in snap_dicts])
    
    info_dicts = []
    
    while not q.empty():
        info_dicts.append(q.get())
      
    fitted_dicts = []
    A_dicts = []
    mu_dicts = []
    sigma_dicts = []
    alpha_dicts = []
    z_T_dicts = []
    
    for info_dict in info_dicts:
        
        fitted_dicts.append(info_dict['snap_dict'])
        A_dicts.append(info_dict['A'])
        mu_dicts.append(info_dict['mu'])
        sigma_dicts.append(info_dict['sigma'])
        alpha_dicts.append(info_dict['alpha'])
        z_T_dicts.append(info_dict['z_T'])
        
    num_fitted_snaps = len(fitted_dicts)
        
    fitted_indices = []
    redshifts = []
    times = []

    halo_masses = []

    SFRs_10 = []
    SFRs_100 = []
    SFRs_1000 = []

    velocities_mass = []
    velocities_vol = []
    velocities_rms_mass = []
    velocities_rms_vol = []
    velocities_spread = []

    sounds_mass = []
    sounds_vol = []
    sounds_rms_mass = []
    sounds_rms_vol = []
    sounds_spread = []

    thermals_mass = []
    thermals_vol = []
    thermals_rms_mass = []
    thermals_rms_vol = []
    thermals_spread = []

    mach_numbers_mass = []
    mach_numbers_vol = []
    mach_numbers_rms_mass = []
    mach_numbers_rms_vol = []
    mach_numbers_spread = []

    As = {}
    mus = {}
    sigmas = {}
    alphas = {}
    z_Ts = {}   
    
    for m in metals:
        As[m] = []
        mus[m] = []
        sigmas[m] = []
        alphas[m] = []
        z_Ts[m] = []

    for i in range(0, num_fitted_snaps):
        
        fitted_dict = fitted_dicts[i]
        A_dict = A_dicts[i]
        mu_dict = mu_dicts[i]
        sigma_dict = sigma_dicts[i]
        alpha_dict = alpha_dicts[i]
        z_T_dict = alpha_dicts[i]
        
        fitted_indices.append(fitted_dict['snap'])
        
        redshifts.append(fitted_dict['redshift'])
        times.append(fitted_dict['time'])
        
        halo_masses.append(fitted_dict['halo_mass'])
        
        SFRs_10.append(fitted_dict['SFR@10Myr'])
        SFRs_100.append(fitted_dict['SFR@100Myr'])
        SFRs_1000.append(fitted_dict['SFR@1000Myr'])

        velocities_mass.append(fitted_dict['velocity_mass'])
        velocities_vol.append(fitted_dict['velocity_vol'])
        velocities_rms_mass.append(fitted_dict['velocity_rms_mass'])
        velocities_rms_vol.append(fitted_dict['velocity_rms_vol'])
        velocities_spread.append(fitted_dict['velocity_spread'])

        sounds_mass.append(fitted_dict['sound_mass'])
        sounds_vol.append(fitted_dict['sound_vol'])
        sounds_rms_mass.append(fitted_dict['sound_rms_mass'])
        sounds_rms_vol.append(fitted_dict['sound_rms_vol'])
        sounds_spread.append(fitted_dict['sound_spread'])

        thermals_mass.append(fitted_dict['thermal_mass'])
        thermals_vol.append(fitted_dict['thermal_vol'])
        thermals_rms_mass.append(fitted_dict['thermal_rms_mass'])
        thermals_rms_vol.append(fitted_dict['thermal_rms_vol'])
        thermals_spread.append(fitted_dict['thermal_spread'])

        mach_numbers_mass.append(fitted_dict['mach_number_mass'])
        mach_numbers_vol.append(fitted_dict['mach_number_vol'])
        mach_numbers_rms_mass.append(fitted_dict['mach_number_rms_mass'])
        mach_numbers_rms_vol.append(fitted_dict['mach_number_rms_vol'])
        mach_numbers_spread.append(fitted_dict['mach_number_spread'])
        
        for m in metals:
            
            As[m].append(A_dict[m])
            mus[m].append(mu_dict[m])
            sigmas[m].append(sigma_dict[m])
            alphas[m].append(alpha_dict[m])
            z_Ts[m].append(z_T_dict[m])
            
    fitted_stats_dict = {'snap': fitted_indices,  'redshift': redshifts, 'time': times, 
                           'halo_mass': halo_masses,
                           'SFR@10Myr': SFRs_10, 'SFR@100Myr': SFRs_100, 'SFR@1000Myr': SFRs_1000,
                           'velocity_mass': velocities_mass, 'velocity_vol': velocities_vol, 
                           'velocity_rms_mass': velocities_rms_mass, 'velocity_rms_vol': velocities_rms_vol, 
                           'velocity_spread': velocities_spread,
                           'sound_mass': sounds_mass, 'sound_vol': sounds_vol,
                           'sound_rms_mass': sounds_rms_mass, 'sound_rms_vol': sounds_rms_vol,
                           'sound_spread': sounds_spread,
                           'thermal_mass': thermals_mass, 'thermal_vol': thermals_vol,
                           'thermal_rms_mass': thermals_rms_mass, 'thermal_rms_vol': thermals_rms_vol,
                           'thermal_spread': thermals_spread,
                           'mach_number_mass': mach_numbers_mass, 'mach_number_vol': mach_numbers_vol,
                           'mach_number_rms_mass': mach_numbers_rms_mass, 
                           'mach_number_rms_vol': mach_numbers_rms_vol, 
                           'mach_number_spread': mach_numbers_spread}

    fitted_stats_df = pd.DataFrame(fitted_stats_dict)
    fitted_stats_df = fitted_stats_df.sort_values(by = 'snap')
    fitted_stats_df = fitted_stats_df.reset_index(drop = True)
    fitted_stats_df.to_csv(sdir + 'fitted_snap_stats.csv', index = False)

    print(fitted_stats_df)
    print()

    for m in metals:
        param_df_init = pd.read_csv(spath_metals[m] + 'data/fit/fit_{}_params.csv'.format(m))
        param_dict_append = {'A': As[m], 'mu': mus[m], 'sigma': sigmas[m], 
                      'alpha': alphas[m], 'z_T': z_Ts[m]}
        param_df_append = pd.DataFrame(param_dict_append)
        param_df = pd.concat([param_df_init, param_df_append], axis = 1)
        param_df.to_csv(spath_metals[m] + 'data/fit/fit_{}_params.csv'.format(m), index = False)
        
    print("All done!")

About to start:


/Users/thepoetoftwilight/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/thepoetoftwilight/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/thepoetoftwilight/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':
/Users/thepoetoftwilight/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in multiply
  if __name__ == '__main__':
/Users/thepoetoftwilight/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in multiply
  if __name__ == '__main__':
/Users/thepoetoftwilight/opt/anaconda3/lib/python

     snap      redshift       time     halo_mass  SFR@10Myr  SFR@100Myr  \
0      79  4.384615e+00   1.420917  5.721844e+09   0.031797    0.017922   
1      80  4.338983e+00   1.439067  5.736586e+09   0.020477    0.020473   
2      81  4.294117e+00   1.457291  5.714834e+09   0.006320    0.021094   
3      82  4.250000e+00   1.475588  5.691378e+09   0.017575    0.017569   
4      83  4.206611e+00   1.493956  5.652333e+09   0.001056    0.016594   
..    ...           ...        ...           ...        ...         ...   
511   596  6.373059e-04  13.789874  4.589969e+10   0.000000    0.000000   
512   597  4.779283e-04  13.792092  4.590174e+10   0.000000    0.000000   
513   598  3.186015e-04  13.794310  4.590417e+10   0.000000    0.000000   
514   599  1.592253e-04  13.796529  4.590615e+10   0.000000    0.000000   
515   600  4.440892e-16  13.798747  4.590833e+10   0.000000    0.000000   

     SFR@1000Myr  velocity_mass  velocity_vol  velocity_rms_mass  ...  \
0       0.012866      82.5